# Proyect Wyscout


In [1]:
import pandas as pd
import glob
import os
import unicodedata
import re

pd.options.display.max_columns = None

## Teams


In [2]:
# Ruta relativa desde la carpeta scripts
# folder_path = '../data/input/Betplay/'
folder_path = '../data/input/LigaEcu/teams/'
# folder_path = '../data/input/Argentina_B/teams/'

# Obtener lista de archivos Excel
excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))

# Leer todos los archivos
dfs = [pd.read_excel(file) for file in excel_files]

# Concatenar todos los DataFrames en uno
combined_df = pd.concat(dfs, ignore_index=True)

# Ver ejemplo
print(f'{len(dfs)} archivos leídos')

16 archivos leídos


In [3]:
# Ruta relativa desde la carpeta scripts
folder_path = '../data/input/LigaEcu/teams/'
# folder_path = '../data/input/Betplay/teams/'
# folder_path = '../data/input/Argentina_B/teams/'

# Obtener lista de archivos Excel
excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))

# Leer todos los archivos y almacenarlos en una lista
dfs = []
for file in excel_files:
    df = pd.read_excel(file)
    print(f"Archivo: {os.path.basename(file)} - Número de columnas: {df.shape[1]}")
    dfs.append(df)

# Concatenar todos los DataFrames en uno
combined_df = pd.concat(dfs, ignore_index=True)

# Ver ejemplo
print(f'{len(dfs)} archivos leídos')


Archivo: Team Stats Universidad Católica.xlsx - Número de columnas: 109
Archivo: Team Stats Técnico Universitario.xlsx - Número de columnas: 109
Archivo: Team Stats Mushuc Runa.xlsx - Número de columnas: 109
Archivo: Team Stats Orense.xlsx - Número de columnas: 109
Archivo: Team Stats Manta.xlsx - Número de columnas: 109
Archivo: Team Stats Macará.xlsx - Número de columnas: 109
Archivo: Team Stats LDU Quito.xlsx - Número de columnas: 109
Archivo: Team Stats Libertad.xlsx - Número de columnas: 109
Archivo: Team Stats Independiente del Valle.xlsx - Número de columnas: 109
Archivo: Team Stats El Nacional.xlsx - Número de columnas: 109
Archivo: Team Stats Deportivo Cuenca.xlsx - Número de columnas: 109
Archivo: Team Stats Delfin.xlsx - Número de columnas: 109
Archivo: Team Stats Emelec.xlsx - Número de columnas: 109
Archivo: Team Stats Barcelona.xlsx - Número de columnas: 109
Archivo: Team Stats Aucas.xlsx - Número de columnas: 109
Archivo: Team Stats Vinotinto de Ecuador.xlsx - Número de 

In [7]:
combined_df.head(5)
combined_df['Competición'].value_counts()
print(combined_df['Competición'].unique())
liga = combined_df['Competición'].unique()[2]
print(f'Competición: {liga}')

[nan 'South America. CONMEBOL Sudamericana' 'Ecuador. Liga Pro'
 'Ecuador. Liga Pro Serie B' 'South America. Copa Libertadores'
 'World. Club Friendlies']
Competición: Ecuador. Liga Pro


In [8]:
df = combined_df.copy()

df = df.dropna(subset=['Partido'])

def rename_columns_with_slash(columns):
    new_columns = []
    i = 0
    while i < len(columns):
        col = columns[i]

        # Si tiene "/", expandimos usando el número de partes
        if "/" in col:
            parts = [p.strip() for p in col.split("/")]
            base = parts[0]
            for j in range(len(parts)):
                if i + j < len(columns):
                    name = base if j == 0 else f"{base} {parts[j]}"
                    new_columns.append(name)
            i += len(parts)
        # Si es 'Unnamed', intenta continuar desde antes
        elif "Unnamed" in col:
            # Si no hemos expandido antes, darle nombre genérico
            new_columns.append(f"%_{parts[0]}")
            i += 1
        else:
            new_columns.append(col)
            i += 1

    # Rellenar si quedaron columnas faltantes
    while len(new_columns) < len(columns):
        new_columns.append(f"col_{len(new_columns)}")

    return new_columns

# Aplicar al DataFrame
df.columns = rename_columns_with_slash(list(df.columns))

df['Fecha'].value_counts().sort_index(ascending=False)
df['Competición'].value_counts().sort_index(ascending=False)

# df = df[df['Competición'] == 'Colombia. Liga BetPlay']
df = df[df['Competición'] == liga]

df = df[df['Fecha'] >= '2025-01-01']

# df.drop_duplicates(inplace=True)

df = df.drop_duplicates(subset=['Fecha', 'Partido', 'Competición', 'Equipo'])

print(df.shape)

df.head(2)

(384, 109)


,Fecha,Partido,Competición,Duración,Equipo,Seleccionar esquema,Goles,xG,Tiros,Tiros a la portería,%_Tiros,Pases,Pases logrados,%_Pases,"Posesión del balón, %",Balones perdidos,Balones perdidos bajos,Balones perdidos medios,Balones perdidos altos,Balones recuperados,Balones recuperados bajos,Balones recuperados medios,Balones recuperados altos,Duelos,Duelos ganados,%_Duelos,Tiros de fuera del área,Tiros de fuera del área a la portería,%_Tiros de fuera del área,Ataques posicionales,Ataques posicionales con remate,%_Ataques posicionales,Contraataques,Contraataques con remate,%_Contraataques,Jugadas a balón parado,Jugadas a balón parado con remate,%_Jugadas a balón parado,Córneres,Córneres con remate,%_Córneres,Tiros libres,Tiros libres con remate,%_Tiros libres,Penaltis,Penaltis marcados,%_Penaltis,Centros,Centros precisos,%_Centros,Pases cruzados en profundidad completados,Pases en profundidad completados,Entradas al área de penalti (carreras,Entradas al área de penalti (carreras pases cruzados),%_Entradas al área de penalti (carreras,Toques en el área de penalti,Duelos ofensivos,Duelos ofensivos ganados,%_Duelos ofensivos,Fuera de juego,Goles recibidos,Tiros en contra,Tiros en contra a la portería,%_Tiros en contra,Duelos defensivos,Duelos defensivos ganados,%_Duelos defensivos,Duelos aéreos,Duelos aéreos ganados,%_Duelos aéreos,Entradas a ras de suelo,Entradas a ras de suelo logradas,%_Entradas a ras de suelo,Interceptaciones,Despejes,Faltas,Tarjetas amarillas,Tarjetas rojas,Pases hacia adelante,Pases hacia adelante logrados,%_Pases hacia adelante,Pases hacia atrás,Pases hacia atrás logrados,%_Pases hacia atrás,Pases laterales,Pases laterales logrados,%_Pases laterales,Pases largos,Pases largos logrados,%_Pases largos,Pases en el último tercio,Pases en el último tercio logrados,%_Pases en el último tercio,Pases progresivos,Pases progresivos precisos,%_Pases progresivos,Desmarques,Desmarques logrados,%_Desmarques,Saques laterales,Saques laterales logrados,%_Saques laterales,Saques de meta,Intensidad de paso,Promedio pases por posesión del balón,Lanzamiento largo %,Distancia media de tiro,Longitud media pases,PPDA
4,2025-08-09,Orense - Universidad Católica 1:1,Ecuador. Liga Pro,100.0,Universidad Católica,4-1-4-1 (100.0%),1.0,1.71,12.0,4.0,33.33,409.0,347.0,84.84,62.25,91.0,11.0,22.0,58.0,78.0,36.0,27.0,15.0,194.0,88.0,45.36,6.0,1.0,16.67,32.0,4.0,12.50,2.0,1.0,50.0,24.0,4.0,16.67,5.0,1.0,20.0,3.0,1.0,33.33,0.0,0.0,0.0,8.0,3.0,37.50,2.0,15.0,33.0,4.0,4.0,23.0,84.0,25.0,29.76,1.0,1.0,5.0,2.0,40.00,65.0,43.0,66.15,18.0,10.0,55.56,1.0,1.0,100.0,31.0,4.0,17.0,5.0,0.0,125.0,99.0,79.20,55.0,50.0,90.91,174.0,154.0,88.51,41.0,23.0,56.1,69.0,52.0,75.36,70.0,58.0,82.86,3.0,1.0,33.33,12.0,12.0,100.00,4.0,15.32,4.54,10.02,18.82,19.61,4.74
5,2025-08-09,Orense - Universidad Católica 1:1,Ecuador. Liga Pro,100.0,Orense,4-4-2 (69.03%),1.0,0.49,5.0,2.0,40.00,241.0,171.0,70.95,37.75,93.0,21.0,32.0,40.0,64.0,41.0,17.0,6.0,194.0,98.0,50.52,4.0,1.0,25.00,19.0,2.0,10.53,0.0,0.0,0.0,27.0,2.0,7.41,4.0,1.0,25.0,2.0,0.0,0.00,0.0,0.0,0.0,14.0,1.0,7.14,0.0,3.0,13.0,3.0,6.0,10.0,65.0,22.0,33.85,3.0,1.0,12.0,4.0,33.33,84.0,59.0,70.24,18.0,6.0,33.33,2.0,1.0,50.0,42.0,6.0,12.0,2.0,1.0,98.0,58.0,59.18,37.0,34.0,91.89,76.0,60.0,78.95,42.0,21.0,50.0,34.0,19.0,55.88,57.0,32.0,56.14,0.0,0.0,0.00,14.0,12.0,85.71,7.0,14.89,2.74,17.43,19.47,22.36,8.93


In [9]:
df['Equipo'].unique()

array(['Universidad Católica', 'Orense', 'Aucas', 'Macará',
       'Vinotinto de Ecuador', 'Manta', 'Mushuc Runa',
       'Independiente del Valle', 'Emelec', 'El Nacional',
       'Deportivo Cuenca', 'LDU Quito', 'Delfin', 'Barcelona',
       'Técnico Universitario', 'Libertad'], dtype=object)

In [10]:
# 1. Columnas identificadoras y métricas
id_cols = ['Fecha', 'Partido', 'Equipo']
metric_cols = [col for col in df.columns if col not in id_cols]

# 2. Preparar el DataFrame contrario
df_contra = df.copy()
df_contra = df_contra.rename(
    columns={**{col: f"{col}_contra" for col in metric_cols}, 'Equipo': 'Equipo_contra'}
)

# 3. Hacer merge por Fecha y Partido (equipo contrario)
df_full = df.merge(
    df_contra,
    on=['Fecha', 'Partido'],
    suffixes=('', '_drop')
)

# 4. Filtrar para evitar unir el equipo consigo mismo
df_full = df_full[df_full['Equipo'] != df_full['Equipo_contra']]

# 5. Eliminar columnas "_drop" si se generaron
df_full = df_full.drop(columns=[col for col in df_full.columns if col.endswith('_drop')])

# 6. Resetear índice
df_full = df_full.reset_index(drop=True)

df_full.drop_duplicates(inplace=True)

df_full.shape

# Detectar si el Equipo es el que inicia el nombre del Partido → Local
df_full['Tipo'] = df_full.apply(
    lambda row: 'Local' if row['Partido'].startswith(row['Equipo']) else 'Visitante',
    axis=1
)


In [11]:
def limpiar_columna(col):
    # Convertir a minúsculas
    col = col.lower()
    # Eliminar tildes
    col = unicodedata.normalize('NFKD', col).encode('ascii', 'ignore').decode('utf-8')
    # Reemplazar espacios por guión bajo
    col = col.replace(" ", "_")
    # Quitar caracteres no alfanuméricos ni _
    col = re.sub(r"[^\w_]", "", col)
    # Eliminar guiones bajos al inicio
    col = re.sub(r"^_+", "%", col)
    return col

# Aplicar a todas las columnas
df_full.columns = [limpiar_columna(col) for col in df_full.columns]

In [12]:
df_full.columns

Index(['fecha', 'partido', 'competicion', 'duracion', 'equipo',
       'seleccionar_esquema', 'goles', 'xg', 'tiros', 'tiros_a_la_porteria',
       ...
       'saques_laterales_logrados_contra', '%saques_laterales_contra',
       'saques_de_meta_contra', 'intensidad_de_paso_contra',
       'promedio_pases_por_posesion_del_balon_contra',
       'lanzamiento_largo__contra', 'distancia_media_de_tiro_contra',
       'longitud_media_pases_contra', 'ppda_contra', 'tipo'],
      dtype='object', length=217)

In [13]:
ligue = df_full['competicion'].unique()[0]
# df_kk = df_full[df_full['Equipo'] == 'Alianza']

# Ordenar por equipo y fecha
df_full.sort_values(by=["equipo", "fecha"], inplace=True)

# Crear columna 'jornada'
df_full["jornada"] = df_full.groupby("equipo").cumcount() + 1

df_full.to_excel(f"../data/output/{ligue}.xlsx", index=False)
df_full.to_csv(f"../data/output/{ligue}.csv", index=False)
# df_kk[['Fecha', 'Partido', 'Equipo', 'Equipo_contra', 'Tiros','Tiros_contra','xG_contra', 'xG']]

# df_kk['xG_contra'].mean()

In [23]:
df_full[df_full['partido']=='Junior - América de Cali 0:1'].head(10)

,fecha,partido,competicion,duracion,equipo,seleccionar_esquema,goles,xg,tiros,tiros_a_la_porteria,%tiros,pases,pases_logrados,%pases,posesion_del_balon_,balones_perdidos,balones_perdidos_bajos,balones_perdidos_medios,balones_perdidos_altos,balones_recuperados,balones_recuperados_bajos,balones_recuperados_medios,balones_recuperados_altos,duelos,duelos_ganados,%duelos,tiros_de_fuera_del_area,tiros_de_fuera_del_area_a_la_porteria,%tiros_de_fuera_del_area,ataques_posicionales,ataques_posicionales_con_remate,%ataques_posicionales,contraataques,contraataques_con_remate,%contraataques,jugadas_a_balon_parado,jugadas_a_balon_parado_con_remate,%jugadas_a_balon_parado,corneres,corneres_con_remate,%corneres,tiros_libres,tiros_libres_con_remate,%tiros_libres,penaltis,penaltis_marcados,%penaltis,centros,centros_precisos,%centros,pases_cruzados_en_profundidad_completados,pases_en_profundidad_completados,entradas_al_area_de_penalti_carreras,entradas_al_area_de_penalti_carreras_pases_cruzados,%entradas_al_area_de_penalti_carreras,toques_en_el_area_de_penalti,duelos_ofensivos,duelos_ofensivos_ganados,%duelos_ofensivos,fuera_de_juego,goles_recibidos,tiros_en_contra,tiros_en_contra_a_la_porteria,%tiros_en_contra,duelos_defensivos,duelos_defensivos_ganados,%duelos_defensivos,duelos_aereos,duelos_aereos_ganados,%duelos_aereos,entradas_a_ras_de_suelo,entradas_a_ras_de_suelo_logradas,%entradas_a_ras_de_suelo,interceptaciones,despejes,faltas,tarjetas_amarillas,tarjetas_rojas,pases_hacia_adelante,pases_hacia_adelante_logrados,%pases_hacia_adelante,pases_hacia_atras,pases_hacia_atras_logrados,%pases_hacia_atras,pases_laterales,pases_laterales_logrados,%pases_laterales,pases_largos,pases_largos_logrados,%pases_largos,pases_en_el_ultimo_tercio,pases_en_el_ultimo_tercio_logrados,%pases_en_el_ultimo_tercio,pases_progresivos,pases_progresivos_precisos,%pases_progresivos,desmarques,desmarques_logrados,%desmarques,saques_laterales,saques_laterales_logrados,%saques_laterales,saques_de_meta,intensidad_de_paso,promedio_pases_por_posesion_del_balon,lanzamiento_largo_,distancia_media_de_tiro,longitud_media_pases,ppda,competicion_contra,duracion_contra,equipo_contra,seleccionar_esquema_contra,goles_contra,xg_contra,tiros_contra,tiros_a_la_porteria_contra,%tiros_contra,pases_contra,pases_logrados_contra,%pases_contra,posesion_del_balon__contra,balones_perdidos_contra,balones_perdidos_bajos_contra,balones_perdidos_medios_contra,balones_perdidos_altos_contra,balones_recuperados_contra,balones_recuperados_bajos_contra,balones_recuperados_medios_contra,balones_recuperados_altos_contra,duelos_contra,duelos_ganados_contra,%duelos_contra,tiros_de_fuera_del_area_contra,tiros_de_fuera_del_area_a_la_porteria_contra,%tiros_de_fuera_del_area_contra,ataques_posicionales_contra,ataques_posicionales_con_remate_contra,%ataques_posicionales_contra,contraataques_contra,contraataques_con_remate_contra,%contraataques_contra,jugadas_a_balon_parado_contra,jugadas_a_balon_parado_con_remate_contra,%jugadas_a_balon_parado_contra,corneres_contra,corneres_con_remate_contra,%corneres_contra,tiros_libres_contra,tiros_libres_con_remate_contra,%tiros_libres_contra,penaltis_contra,penaltis_marcados_contra,%penaltis_contra,centros_contra,centros_precisos_contra,%centros_contra,pases_cruzados_en_profundidad_completados_contra,pases_en_profundidad_completados_contra,entradas_al_area_de_penalti_carreras_contra,entradas_al_area_de_penalti_carreras_pases_cruzados_contra,%entradas_al_area_de_penalti_carreras_contra,toques_en_el_area_de_penalti_contra,duelos_ofensivos_contra,duelos_ofensivos_ganados_contra,%duelos_ofensivos_contra,fuera_de_juego_contra,goles_recibidos_contra,tiros_en_contra_contra,tiros_en_contra_a_la_porteria_contra,%tiros_en_contra_contra,duelos_defensivos_contra,duelos_defensivos_ganados_contra,%duelos_defensivos_contra,duelos_aereos_contra,duelos_aereos_ganados_contra,%duelos_aereos_contra,entradas_a_ras_de_suelo_contra,entradas_a_ras_de_suelo_logradas_contra,%entradas_a_ras_de_suelo_c

In [38]:
# Suma
df_sum_div = df_full.groupby(['equipo', 'tipo'], as_index=False).sum(numeric_only=True)

# Promedio
df_mean_div = df_full.groupby(['equipo', 'tipo'], as_index=False).mean(numeric_only=True)

# Copiar suma
df_per90_div = df_sum_div.copy()

# Columnas a ajustar (todas menos claves)
cols = [col for col in df_per90_div.columns if col not in ['equipo', 'tipo', 'duracion']]

# Duración total por grupo
duracion = df_per90_div['duracion']

# Calcular métricas normalizadas
for col in cols:
    df_per90_div[col] = (90 / duracion) * df_per90_div[col]
    df_per90_div.rename(columns={col: f"{col}_per90"}, inplace=True)

In [ ]:
# Agrupaciones base
df_sum = df_full.groupby("equipo", as_index=False).sum(numeric_only=True)
df_mean = df_full.groupby("equipo", as_index=False).mean(numeric_only=True)

# Copiar df_sum para trabajar sobre él
df_per90 = df_sum.copy()

# Guardar y eliminar duración para el cálculo
duracion_total = df_sum['duracion']

# Aplicar fórmula por columna numérica (excepto 'duracion')
cols_to_adjust = [col for col in df_per90.columns if col not in ['equipo', 'duracion']]

for col in cols_to_adjust:
    df_per90[col] = (90 / duracion_total) * df_per90[col]


In [23]:
df_per90[df_per90['equipo']=='Alianza'].head(2)

,equipo,duracion,goles,xg,tiros,tiros_a_la_porteria,%tiros,pases,pases_logrados,%pases,posesion_del_balon_,balones_perdidos,balones_perdidos_bajos,balones_perdidos_medios,balones_perdidos_altos,balones_recuperados,balones_recuperados_bajos,balones_recuperados_medios,balones_recuperados_altos,duelos,duelos_ganados,%duelos,tiros_de_fuera_del_area,tiros_de_fuera_del_area_a_la_porteria,%tiros_de_fuera_del_area,ataques_posicionales,ataques_posicionales_con_remate,%ataques_posicionales,contraataques,contraataques_con_remate,%contraataques,jugadas_a_balon_parado,jugadas_a_balon_parado_con_remate,%jugadas_a_balon_parado,corneres,corneres_con_remate,%corneres,tiros_libres,tiros_libres_con_remate,%tiros_libres,penaltis,penaltis_marcados,%penaltis,centros,centros_precisos,%centros,pases_cruzados_en_profundidad_completados,pases_en_profundidad_completados,entradas_al_area_de_penalti_carreras,entradas_al_area_de_penalti_carreras_pases_cruzados,%entradas_al_area_de_penalti_carreras,toques_en_el_area_de_penalti,duelos_ofensivos,duelos_ofensivos_ganados,%duelos_ofensivos,fuera_de_juego,goles_recibidos,tiros_en_contra,tiros_en_contra_a_la_porteria,%tiros_en_contra,duelos_defensivos,duelos_defensivos_ganados,%duelos_defensivos,duelos_aereos,duelos_aereos_ganados,%duelos_aereos,entradas_a_ras_de_suelo,entradas_a_ras_de_suelo_logradas,%entradas_a_ras_de_suelo,interceptaciones,despejes,faltas,tarjetas_amarillas,tarjetas_rojas,pases_hacia_adelante,pases_hacia_adelante_logrados,%pases_hacia_adelante,pases_hacia_atras,pases_hacia_atras_logrados,%pases_hacia_atras,pases_laterales,pases_laterales_logrados,%pases_laterales,pases_largos,pases_largos_logrados,%pases_largos,pases_en_el_ultimo_tercio,pases_en_el_ultimo_tercio_logrados,%pases_en_el_ultimo_tercio,pases_progresivos,pases_progresivos_precisos,%pases_progresivos,desmarques,desmarques_logrados,%desmarques,saques_laterales,saques_laterales_logrados,%saques_laterales,saques_de_meta,intensidad_de_paso,promedio_pases_por_posesion_del_balon,lanzamiento_largo_,distancia_media_de_tiro,longitud_media_pases,ppda,duracion_contra,goles_contra,xg_contra,tiros_contra,tiros_a_la_porteria_contra,%tiros_contra,pases_contra,pases_logrados_contra,%pases_contra,posesion_del_balon__contra,balones_perdidos_contra,balones_perdidos_bajos_contra,balones_perdidos_medios_contra,balones_perdidos_altos_contra,balones_recuperados_contra,balones_recuperados_bajos_contra,balones_recuperados_medios_contra,balones_recuperados_altos_contra,duelos_contra,duelos_ganados_contra,%duelos_contra,tiros_de_fuera_del_area_contra,tiros_de_fuera_del_area_a_la_porteria_contra,%tiros_de_fuera_del_area_contra,ataques_posicionales_contra,ataques_posicionales_con_remate_contra,%ataques_posicionales_contra,contraataques_contra,contraataques_con_remate_contra,%contraataques_contra,jugadas_a_balon_parado_contra,jugadas_a_balon_parado_con_remate_contra,%jugadas_a_balon_parado_contra,corneres_contra,corneres_con_remate_contra,%corneres_contra,tiros_libres_contra,tiros_libres_con_remate_contra,%tiros_libres_contra,penaltis_contra,penaltis_marcados_contra,%penaltis_contra,centros_contra,centros_precisos_contra,%centros_contra,pases_cruzados_en_profundidad_completados_contra,pases_en_profundidad_completados_contra,entradas_al_area_de_penalti_carreras_contra,entradas_al_area_de_penalti_carreras_pases_cruzados_contra,%entradas_al_area_de_penalti_carreras_contra,toques_en_el_area_de_penalti_contra,duelos_ofensivos_contra,duelos_ofensivos_ganados_contra,%duelos_ofensivos_contra,fuera_de_juego_contra,goles_recibidos_contra,tiros_en_contra_contra,tiros_en_contra_a_la_porteria_contra,%tiros_en_contra_contra,duelos_defensivos_contra,duelos_defensivos_ganados_contra,%duelos_defensivos_contra,duelos_aereos_contra,duelos_aereos_ganados_contra,%duelos_aereos_contra,entradas_a_ras_de_suelo_contra,entradas_a_ras_de_suelo_logradas_contra,%entradas_a_ras_de_suelo_contra,interceptaciones_contra,despejes_contra,faltas_contra,tarjetas_amarillas_contra,tarjetas_rojas_contr

In [40]:
# 1. Filtrar las columnas necesarias
df_xg = df_full[["fecha", "equipo", "xg", "xg_contra"]].copy()

# 2. Ordenar por Equipo y Fecha (ascendente)
df_xg.sort_values(by=["equipo", "fecha"], inplace=True)

# 3. Enumerar por equipo (índice consecutivo por grupo)
df_xg["jornada"] = df_xg.groupby("equipo").cumcount() + 1

# 4. Eliminar la columna Fecha
df_xg.drop(columns=["fecha"], inplace=True)

# 5. (Opcional) Reordenar columnas si deseas
df_xg = df_xg[["equipo", "jornada", "xg", "xg_contra"]]

In [10]:
print(df_full['equipo'].value_counts())

equipo
Atlético Nacional       36
Junior                  33
Medellín                32
Santa Fe                32
Millonarios             31
Once Caldas             31
América de Cali         31
Tolima                  30
Deportivo Cali          24
Alianza                 24
Deportivo Pereira       24
Boyacá Chicó            24
Águilas Doradas         24
Llaneros                24
Fortaleza               24
La Equidad              24
Envigado                23
Atlético Bucaramanga    22
Unión Magdalena         22
Deportivo Pasto         21
Name: count, dtype: int64


In [ ]:
Chaco For Ever            22
Central Norte             22
Defensores Unidos         22
Temperley                 22
Nueva Chicago             21
Agropecuario              21
Club Atlético Mitre       21
Colón                     21
Defensores de Belgrano    21
Chacarita Juniors         21
Gimnasia Jujuy            21
Estudiantes Río Cuarto    21
Almirante Brown           20
Talleres Remedios         20
Deportivo Morón           19
Estudiantes Caseros       19
Gimnasia Mendoza          19
San Telmo                 19

## Players


In [ ]:
_folder_path_players = '../data/input/LigaEcu/players/'

df_players = pd.read_excel(_folder_path_players + 'players_ecu.xlsx')